## Dosya Dil Çeviri Aracı (File Language Translation Tool)

### Kütüphaneleri yükleme

In [36]:
import pandas as pd
import docx
from googletrans import Translator
from langdetect import detect
import os

### Dil isimleri ile kısaltmalar arasında eşleme

In [37]:
dil_eslemesi = {
    'Türkçe': 'tr',
    'İngilizce': 'en',
    'İspanyolca': 'es',
    'Fransızca': 'fr',
    'Almanca': 'de',
    'İtalyanca': 'it',
    'Portekizce': 'pt',
    'Arapça': 'ar',
    'Çince': 'zh-cn',
    'Japonca': 'ja',
    'Korece': 'ko',
    'Rusça': 'ru',
    'Hollandaca': 'nl',
    'İsveççe': 'sv',
    'Danimarkaca': 'da',
    'Norveççe': 'no',
    'Fince': 'fi',
    'Macarca': 'hu',
    'Polonca': 'pl',
    'Çekçe': 'cs',
    'Slovakça': 'sk',
    'Romence': 'ro',
    'Bulgarca': 'bg',
    'Yunanca': 'el',
    'Türkmence': 'tk',
    'Hindice': 'hi',
    'Bengalce': 'bn',
    'Urduca': 'ur',
    'Vietce': 'vi',
    'Thai': 'th',
    'İndonezyaca': 'id',
    'Malayca': 'ms',
    'Filipince': 'tl'
}


### Çeviri aracı oluşturma

In [38]:
cevirmen = Translator()

### Metin çevirme fonksiyonu

In [39]:
def ceviri_metni(text, hedef_dil):
    if not text.strip():
        return text
    try:
        algilanan_dil = detect(text)  # Metnin dilini tespit etme
        cevrilen = cevirmen.translate(text, src=algilanan_dil, dest=hedef_dil)  # Metni hedef dile çevirme
        return cevrilen.text
    except Exception as e:
        print(f"Metin çevrilirken hata oluştu: {text[:100]}, hata: {e}")
        return text  # Hata durumunda orijinal metni geri döndürme

### CSV dosyasını çevirme fonksiyonu

In [40]:
def ceviri_csv(dosya_yolu, hedef_dil):
    try:
        df = pd.read_csv(dosya_yolu)  # CSV dosyasını okuma
    except Exception as e:
        print(f"CSV dosyasını okurken hata oluştu: {e}")
        return
    cevirilecek_sutunlar = df.columns
    for sutun_adi in cevirilecek_sutunlar:
        if sutun_adi in df.columns:
            print(f"Sütunu çevir: {sutun_adi}")
            df[sutun_adi] = df[sutun_adi].apply(lambda x: ceviri_metni(x, hedef_dil) if isinstance(x, str) else x)  # Metinleri çevirme
    
    yeni_dosya_yolu = dosya_yolu.replace('.csv', f'_cevrildi_{hedef_dil}.csv')  # Yeni dosya ismini oluşturma
    df.to_csv(yeni_dosya_yolu, index=False)  # Çevrilen veriyi yeni dosyaya kaydetme
    print(f"Dosya çevrildi: {yeni_dosya_yolu}")

### TXT dosyasını çevirme fonksiyonu

In [41]:
def ceviri_txt(dosya_yolu, hedef_dil):
    try:
        with open(dosya_yolu, 'r', encoding='utf-8') as dosya:
            metin = dosya.read()  # TXT dosyasını okuma
    except Exception as e:
        print(f"TXT dosyasını okurken hata oluştu: {e}")
        return
    cevrilen_metin = ceviri_metni(metin, hedef_dil)  # Metni çevirme
    
    yeni_dosya_yolu = dosya_yolu.replace('.txt', f'_cevrildi_{hedef_dil}.txt')  # Yeni dosya ismini oluşturma
    try:
        with open(yeni_dosya_yolu, 'w', encoding='utf-8') as dosya:
            dosya.write(cevrilen_metin)  # Çevrilen metni yeni dosyaya yazma
    except Exception as e:
        print(f"TXT dosyası yazılırken hata oluştu: {e}")
        return
    print(f"Dosya çevrildi: {yeni_dosya_yolu}")

### DOCX dosyasını çevirme fonksiyonu

In [42]:
def ceviri_docx(dosya_yolu, hedef_dil):
    try:
        doc = docx.Document(dosya_yolu)  # DOCX dosyasını okuma
    except Exception as e:
        print(f"DOCX dosyasını okurken hata oluştu: {e}")
        return
    for paragraf in doc.paragraphs:
        if paragraf.text.strip():  # Boş olmayan metinler için çeviri yapma
            paragraf.text = ceviri_metni(paragraf.text, hedef_dil)
    
    yeni_dosya_yolu = dosya_yolu.replace('.docx', f'_cevrildi_{hedef_dil}.docx')  # Yeni dosya ismini oluşturma
    try:
        doc.save(yeni_dosya_yolu)  # Çevrilen metni yeni dosyaya kaydetme
    except Exception as e:
        print(f"DOCX dosyası kaydedilirken hata oluştu: {e}")
        return
    print(f"Dosya çevrildi: {yeni_dosya_yolu}")

### Ana fonksiyon

In [43]:
def ana_fonksiyon():
    dosya_yolu = input("Çevrilecek dosyanın yolunu giriniz (csv, txt, docx): ").strip()  # Dosya yolunu kullanıcıdan alma
    hedef_dil_adi = input("Hedef dili giriniz ('Türkçe', 'İngilizce'): ").strip()  # Hedef dili kullanıcıdan alma
    
    hedef_dil = dil_eslemesi.get(hedef_dil_adi)  # Hedef dilin kısaltmasını alma
    if not hedef_dil:
        print("Desteklenmeyen hedef dil.")
        return

    if not os.path.isfile(dosya_yolu):
        print("Dosya bulunamadı.")
        return

    dosya_uzantisi = os.path.splitext(dosya_yolu)[1].lower()  # Dosya uzantısını alma
    if dosya_uzantisi == '.csv':
        ceviri_csv(dosya_yolu, hedef_dil)  # CSV dosyasını çevirme
    elif dosya_uzantisi == '.txt':
        ceviri_txt(dosya_yolu, hedef_dil)  # TXT dosyasını çevirme
    elif dosya_uzantisi == '.docx':
        ceviri_docx(dosya_yolu, hedef_dil)  # DOCX dosyasını çevirme
    else:
        print("Desteklenmeyen dosya formatı.")

### Ana fonksiyonu çalıştırma

In [44]:
if __name__ == "__main__":
    ana_fonksiyon()

Çevrilecek dosyanın yolunu giriniz (csv, txt, docx): csvMetni.csv
Hedef dili giriniz ('Türkçe', 'İngilizce'): Türkçe
Sütunu çevir: ArticleId
Sütunu çevir: Text
Sütunu çevir: Category
Dosya çevrildi: csvMetni_cevrildi_tr.csv
